# Hyperspectral CNN

This code is to train the Hyperspectral CNN. Warning: You need at least 18GB of RAM, to process the TfRecords.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from __future__ import annotations
from collections.abc import Mapping

import tensorflow as tf
import numpy as np

class TfrecordHelper():
    def __init__(self, path: str, ls_bands = "ms", nl_band = None):
        """
        Init function for creating the TfrecordHelper object.

        Args:
        - path (str): Path where tfrecord file is located.
        - ls_bands (str): Select landsat bands, "ms" (default) for all bands or "rgb" for RED, BLUE, GREEN bands.
        - nl_bands (str): For including the nightlight band, set any other value then None (default).

        """

        self.raw_dataset: tf.TFRecordDataset = tf.data.TFRecordDataset(path, compression_type="GZIP")
        self.dataset: tf.TFRecordDataset | None = None
        self.ls_bands: str = ls_bands
        self.nl_band: str | None = nl_band
        self.nbands = 8
        # use the keywords used in the csv file to scrape the tfrecords from gee
        self.keyword_year = "year"
        self.keyword_lat = "lat"
        self.keyword_lon = "lon"
        self.scalar_keys = [self.keyword_lat, self.keyword_lon, self.keyword_year] # used 
        self.means = None
        self.stads = None
    
    def process_dataset(self, normalize = False):
        """
        Method for processing the raw_dataset based on selected bands.
        """
        
        x = np.empty(shape=(255**2))
        x.fill(0)
        def_value = tf.convert_to_tensor(x, tf.float32)

        def process_tfrecord(record: tf.train.Example) -> dict[any, any, any]:
            """
            Helper function for the map call, which processes the each tfrecord (feature).

            Args: 
            - record (tf.train.Example): feature to process
            
            Returns: 
            result (dict[any, any, any]): contains processed feature
            """
            bands = []
            if self.ls_bands == "rgb":
                # bands = ["BLUE", "GREEN", "RED"]  # BGR order
                bands = ["RED", "GREEN", "BLUE"]
            elif self.ls_bands == "ms":
                bands = ["RED", "GREEN", "BLUE", "SWIR1", "SWIR2", "TEMP1", "NIR"]
            if self.nl_band is not None:
                bands += ["NIGHTLIGHTS"]

            keys_to_features = {}
            for band in bands:
                keys_to_features[band] = tf.io.FixedLenFeature(shape=[255**2], dtype=tf.float32, default_value=def_value)
            for key in self.scalar_keys:
                keys_to_features[key] = tf.io.FixedLenFeature(shape=[], dtype=tf.float32)
        
            
           
            # cons_pc = tf.cast(ex.get("cons_pc", -1), tf.float32)

            ex = tf.io.parse_single_example(record, features=keys_to_features)
            loc = tf.stack([ex[self.keyword_lat], ex[self.keyword_lon]])
            year = tf.cast(ex.get("year", -1), tf.int32)
            for band in bands:
                ex[band].set_shape([255 * 255])
                # reshape to (255, 255) and crop to (224, 224)
                ex[band] = tf.reshape(ex[band], [255, 255])[15:-16, 15:-16]
                if normalize:
                    if band == "NIGHTLIGHTS":
                        ex[band] = (ex[band] - self.means["VIIRS"]) / self.stads["VIIRS"]
                    else:
                        ex[band] = (ex[band] - self.means[band]) / self.stads[band]

            img = tf.stack([ex[band] for band in bands], axis=2)
            result = {"images": img, "locs": loc, "years": year}
            return result
        
        self.dataset = self.raw_dataset.map(process_tfrecord, num_parallel_calls=3)
    

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from bisect import bisect
from sklearn.mixture import GaussianMixture as GMM
from torch.optim import lr_scheduler
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from tqdm.notebook import tqdm

import copy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import ResNet18_Weights

In [4]:
torch.__version__
tf.__version__

'2.9.2'

## Load data and preprocess

In [5]:
def load_dataset(path: str):
    """
    Helper to load dataset

    Args:
    - path (str): Path to dataset

    Returns:
    - dic which contains all data
    """
    tf_helper = TfrecordHelper(path, ls_bands="ms", nl_band="viirs")
    input_dic = {}
    tf_helper.keyword_lat = "lat"
    tf_helper.keyword_lon = "lon"
    tf_helper.process_dataset()
    for i, feature in enumerate(tf_helper.dataset):
        input_dic[i] = {
        "year": feature["years"].numpy(),
        "cluster_lat": feature["locs"].numpy()[0],
        "cluster_lon": feature["locs"].numpy()[1],
        "img": (feature["images"][:,:,:7].numpy()),
        "nightlight": np.mean(feature["images"][:,:,7].numpy()),
    }
    
    # Remove data where entry is broken (one channel contains only zeros)
    remove = []
    for feature in tqdm(input_dic):
        if input_dic[feature]["nightlight"] == 0:
            remove.append(feature)
            continue
        for dim in input_dic[feature]["img"]:
            if not np.any(dim):
                remove.append(feature)
                break
    
    for r in remove:
        input_dic.pop(r)
    return input_dic

In [10]:
path = "drive/MyDrive/tfrecords_raw" # path to the processed tfrecords from the previous step
files = list(filter(lambda p : not p.startswith("."), os.listdir(path))) # Ignoring hidden files
print(files)


['TZA_2014_00.tfrecord.gz', 'TZA_2012_00.tfrecord.gz', 'ETH_2013_00.tfrecord.gz', 'NER_2018_00.tfrecord.gz', 'ETH_2015_00.tfrecord.gz', 'NG_2015_00.tfrecord.gz', 'MW_2019_00.tfrecord.gz', 'MW_2016_00.tfrecord.gz', 'MLI_2014_00.tfrecord.gz', 'UGA_2010_00.tfrecord.gz', 'NER_2011_00.tfrecord.gz', 'ETH_2018_00.tfrecord.gz', 'NG_2012_00.tfrecord.gz', 'NG_2018_00.tfrecord.gz', 'UGA_2009_00.tfrecord.gz', 'MLI_2018_00.tfrecord.gz']


In [11]:
input_dics = [] # will contain all information
for file in files:
    raw_path = f'{path}/{file}'
    data = load_dataset(raw_path)
    input_dics.append(data)

  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 0/1611 [00:00<?, ?it/s]

  0%|          | 0/475 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

...

  0%|          | 0/516 [00:00<?, ?it/s]

  0%|          | 0/503 [00:00<?, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

  0%|          | 0/590 [00:00<?, ?it/s]

  0%|          | 0/524 [00:00<?, ?it/s]

In [12]:
X = []
y = []
years = []
lat = []
lon = []
for country in tqdm(input_dics):
    data = country
    for feature in data:
        years.append(data[feature]["year"])
        lat.append(data[feature]["cluster_lat"])
        lon.append(data[feature]["cluster_lon"])
        data[feature]["img"][:3,:,:] *=3 # RGB images to dark, got better performance by using it
        X.append(data[feature]["img"])
        y.append(data[feature]["nightlight"])
X = np.array(X)
y = np.array(y)

  0%|          | 0/16 [00:00<?, ?it/s]

In [13]:
means = [np.mean(X[:,i,:,:]) for i in range(7)]
stds = [np.std(X[:,i,:,:]) for i in range(7)]

In [15]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=means, std=stds)
])

## Bins for nighttime images

In [16]:
def nightlights_to_class(data):
    """
    Data are labels. Perform GMM based on the input and creates 5 classes out of it.

    Args:
    - data: radiance (nighttime images)

    Return:
    - list of labels
    """
    x = data.reshape(-1,1)
    gmm = GMM(n_components=5).fit(x)
    labels = gmm.predict(x)
    cut_label1 = data[labels==0].max()
    cut_label2 = data[labels==1].max()
    cut_label3 = data[labels==2].max()
    cut_label4 = data[labels==3].max()
    cut_label5 = data[labels==4].max()
    cutoffs = [cut_label1, cut_label2, cut_label3,  cut_label4, cut_label5]
    cutoffs = sorted(cutoffs)
    
    y_labels = []
    for d in data:
        if d <= cutoffs[0]:
            y_labels.append(0)
        elif d <= cutoffs[1]:
            y_labels.append(1)
        elif d <= cutoffs[2]:
            y_labels.append(2)
        elif d <= cutoffs[3]:
            y_labels.append(3)
        else:
            y_labels.append(4)
    return np.array(y_labels)

In [17]:
y_labels = nightlights_to_class(y)

## Pytorch Dataset

In [18]:
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        if self.transform:
            #x = self.transform(x.transpose(2, 0, 1)) # transpose is required by PyTorch
            x = self.transform(x) 

        return x, y
    
    def __len__(self):
        return len(self.data)

In [19]:
dataset = MyDataset(X, y_labels, preprocess)

In [20]:
loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

In [21]:
indices = list(range(len(dataset)))
split = int(np.floor(.4 * len(dataset)))
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                                sampler=valid_sampler)
dataloaders = {
    "train": train_loader,
    "val": validation_loader
}

dataset_sizes = {
    "train": len(train_sampler),
    "val": len(valid_sampler)
}

## CNN

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data
                                              )
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=ResNet18_Weights.DEFAULT) # load resnet

Hyperspectral Setting

In [25]:
new_input = nn.Conv2d(7, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), dilation=1, bias=False)
model.conv1 = new_input

Modify outputs

In [26]:
model_ft = model
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

In [27]:
model_ft = model_ft.to(device)

In [28]:
print(model_ft)

ResNet(
  (conv1): Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=31)

Epoch 0/30
----------
train Loss: 0.6684 Acc: 0.7324
val Loss: 1.3431 Acc: 0.3556

Epoch 1/30
----------
train Loss: 0.6581 Acc: 0.7308
val Loss: 1.9432 Acc: 0.4730

Epoch 2/30
----------
train Loss: 0.6423 Acc: 0.7410
val Loss: 1.2550 Acc: 0.3994

Epoch 3/30
----------
train Loss: 0.6052 Acc: 0.7585
val Loss: 1.3896 Acc: 0.3270

Epoch 4/30
----------
train Loss: 0.5700 Acc: 0.7735
val Loss: 1.5860 Acc: 0.4392

Epoch 5/30
----------
train Loss: 0.5426 Acc: 0.7855
val Loss: 2.2840 Acc: 0.4730

Epoch 6/30
----------
train Loss: 0.5234 Acc: 0.7906
val Loss: 2.4830 Acc: 0.4730

Epoch 7/30
----------
train Loss: 0.4408 Acc: 0.8286
val Loss: 1.7788 Acc: 0.3355

Epoch 8/30
----------
train Loss: 0.4161 Acc: 0.8432
val Loss: 1.5605 Acc: 0.5354

Epoch 9/30
----------
train Loss: 0.4072 Acc: 0.8506
val Loss: 2.2550 Acc: 0.3758

Epoch 10/30
----------
train Loss: 0.3963 Acc: 0.8538
val Loss: 2.0133 Acc: 0.3880

Epoch 11/30
----------
train Loss: 0.3970 Acc: 0.8530
val Loss: 1.8407 Acc: 0.4325

Ep

In [34]:
torch.save(model.state_dict(), f'drive/MyDrive/data/weights/model_weights_all_countries_multichannel_{time.time()}.pth')

## Extract Weights

In [35]:
nmodel = torch.nn.Sequential(*list(model_ft.children())[:-1])

In [36]:
if torch.cuda.is_available():
    nmodel.to('cuda')

Forward pass

In [37]:
for data in input_dics:
    for feature in tqdm(data, total=len(data)):
        input_batch = preprocess(data[feature]['img']).unsqueeze(0)
        
        with torch.no_grad():
            output = nmodel(input_batch.to('cuda'))
        data[feature]["feature"] = np.squeeze(output.cpu())

  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/475 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

...

  0%|          | 0/516 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

  0%|          | 0/590 [00:00<?, ?it/s]

  0%|          | 0/524 [00:00<?, ?it/s]

Merge of weights and dataframe

In [38]:
df = pd.DataFrame()
for data in input_dics:
    years = []
    lat = []
    lon = []
    features = []
    nightlights = []
    for feature in tqdm(data, total=len(data)):
        years.append(data[feature]["year"])
        lat.append(data[feature]["cluster_lat"])
        lon.append(data[feature]["cluster_lon"])
        features.append(data[feature]["feature"].numpy().tolist())
        nightlights.append(data[feature]["nightlight"])
    tmp = pd.DataFrame.from_dict({"year": years, "lat": lat, 'lon': lon, "features": features, "nightlight": nightlights})
    df = df.append(tmp)

  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/475 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

...

  0%|          | 0/516 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

  0%|          | 0/590 [00:00<?, ?it/s]

  0%|          | 0/524 [00:00<?, ?it/s]

In [39]:
df.to_csv("drive/MyDrive/data/weights/weights_all_countries.csv", index=False) # path to save file